

docker cp /home/yury/weatherAUS.csv bd_cont:/home/weatherAUS.csv

docker cp /home/yury/normalise.ipynb bd_cont:/home/normalise.ipynb

In [1]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.DoubleType

Intitializing Scala interpreter ...

Spark Web UI available at http://2ff13c8b9cc0:4041
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1590928140956)
SparkSession available as 'spark'


2020-05-31 12:28:59,784 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-05-31 12:29:00,835 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.DoubleType


In [2]:
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("part-00000-dfc11f65-5287-4c73-be9f-d12bac5d8fff-c000.csv")

weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [3]:
weather_org_df.registerTempTable("Aus_Rain_Data")

<font color='red'>
YURY =====================================      MY CODE STARTS     =====================================
 </font>

In [5]:
val df0 = spark.sql("select * from Aus_Rain_Data")

df0: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [6]:
val df1 = df0.withColumn("Pressure9amTmp", df0("Pressure9am").cast(DoubleType))
    .drop("Pressure9am")
    .withColumnRenamed("Pressure9amTmp", "Pressure9am")

val df2 = df1.withColumn("Pressure3pmTmp", df1("Pressure3pm").cast(DoubleType))
    .drop("Pressure3pm")
    .withColumnRenamed("Pressure3pmTmp", "Pressure3pm")

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]
df2: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [7]:
val min_P9am = df2.agg(min("Pressure9am") as "min_P9am").select(col("min_P9am")).first.getDouble(0)
val max_P9am = df2.agg(max("Pressure9am") as "max_P9am").select(col("max_P9am")).first.getDouble(0)

min_P9am: Double = 980.5
max_P9am: Double = 1041.0


In [8]:
val min_P3pm = df2.agg(min("Pressure3pm") as "min_P3pm").select(col("min_P3pm")).first.getDouble(0)
val max_P3pm = df2.agg(max("Pressure3pm") as "max_P3pm").select(col("max_P3pm")).first.getDouble(0)

min_P3pm: Double = 977.1
max_P3pm: Double = 1039.6


In [9]:
val minPRange = 970.0    //from the report
val maxPRange = 1050.0   //from the report

minPRange: Double = 970.0
maxPRange: Double = 1050.0


\begin{equation}
  Rescaled_{9am} = \frac{Pressure_i - Pressure_{min9am}}{Pressure_{max9am} - Pressure_{min9am}} * (maxPressureRange - minPressureRange) + minPressureRange
\end{equation}

In [10]:
val df3 = df2.withColumn("Pressure3pm_Normalised", 
    (((df2("Pressure3pm") - min_P3pm) / (max_P3pm -min_P3pm)) * (maxPRange - minPRange)) + minPRange)

df3: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


In [11]:
val df4 = df3.withColumn("Pressure9am_Normalised", 
    (((df2("Pressure9am") - min_P9am) / (max_P9am -min_P9am)) * (maxPRange - minPRange)) + minPRange)

df4: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 24 more fields]


In [12]:
val df5 = df4.drop("Pressure3pm").drop("Pressure9am")

df5: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [13]:
val df6 = df5.withColumnRenamed("Pressure3pm_Normalised","Pressure3pm")

df6: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [14]:
val df7 = df6.withColumnRenamed("Pressure9am_Normalised","Pressure9am")

df7: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [15]:
df7.coalesce(1)
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("normalised")

2020-05-31 12:30:07,207 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


In [16]:
!ls /home/normalised

_SUCCESS  part-00000-b5e228f4-5e27-425e-bbb0-1f105a85fd47-c000.csv



In [17]:
!mv /home/normalised/*.csv /home/normalised/weatherAUS_N.csv

In [18]:
!mv /home/normalised/weatherAUS_N.csv /home/weatherAUS_N.csv

In [19]:
!wc -l   weatherAUS_N.csv
!wc -l   weatherAUS.csv

142194 weatherAUS_N.csv

142194 weatherAUS.csv



In [20]:
!rm -r /home/normalised/

<font color='red'>
YURY=====================================      MY CODE ENDS     =====================================
 </font>


docker cp  bd_cont:/home/weatherAUS_N.csv /home/yury/weatherAUS_N.csv

docker cp  bd_cont:/home/normalise.ipynb /home/yury/normalise.ipynb
